In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[vector]

In [ ]:
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
# geemap.update_package()
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [ ]:
# define directories
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com/data'

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + '/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract OSM by city

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name'] + '-' + boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = aws_s3_dir + '/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    #query OSM
    ## create bounding box
    bb = boundary_geo_ee.union(1).geometry().buffer(100).bounds(1000)
    print(bb.getInfo())
    ## return the list of coordinates
    listCoords = ee.Array.cat(bb.coordinates(), 1); 
    ##Casting it to an array makes it possible to slice out the x and y coordinates:
    ##get the X-coordinates
    xCoords = listCoords.slice(1, 0, 1); #print('xCoords', xCoords.getInfo());
    yCoords = listCoords.slice(1, 1, 2); #print('yCoords', yCoords.getInfo());
    ## Reducing each array reveals then the min and max values:
    ## reduce the arrays to find the max (or min) value
    West = ee.Number(xCoords.reduce('min', [0]).get([0,0])).getInfo(); 
    East = ee.Number(xCoords.reduce('max', [0]).get([0,0])).getInfo(); 
    North = ee.Number(yCoords.reduce('min', [0]).get([0,0])).getInfo(); 
    South = ee.Number(yCoords.reduce('max', [0]).get([0,0])).getInfo(); 
    
    ## set OSM tags relevant to open space category
    tags = {'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} #{'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} #}

    # use bounding box to get geodataframe of all OSM data on recreation sites/parks. Use .drop to remove properties that have incompatible names with FeatureCollections
    RecSites = geemap.osm_gdf_from_bbox(North, South, East, West, tags)#.drop(columns=['contact:p.o.box']) # use to remove properties that have incompatible names with FeatureCollections
    RecSites = RecSites.reset_index()
    #RecSites.sort_index()#.sample(3)
    #RecSites.sort_values(by=['osmid'],axis='index')
    
    ## keep only columns desired to reduce file size 
    RecSites = RecSites[RecSites['element_type']!= 'node']
    RecSites = RecSites[RecSites.geom_type != 'LineString']
    RecSites = RecSites.loc[:, ['osmid','geometry']] #['element_type','osmid','leisure','boundary','name','geometry']
    #RecSites#.sample(3)
    
    # convert Geodataframe to GeoJson
    RecSitesGJ = geemap.gdf_to_geojson(RecSites)
    
    # convert GeoJson to ee.FeatureCollection
    RecSitesFC = geemap.geojson_to_ee(RecSitesGJ)#.filter(ee.Filter.eq('element_type','way')).select(['osmid','element_type','geometry','leisure','boundary'])
    
    # Download FeatureCollection of OSM data (ways/polygons only) to Google Drive as geojson
    # Set configuration parameters for output vector
    task_config = {
        'folder': 'data',  # output Google Drive folder
        'fileFormat': 'Geojson',
        'description': boundary_id + '-OSM-OpenSpace',
        #'selectors': col_names,  # a list of properties/attributes to be exported
    }

    print('Exporting ' +boundary_id+ '-OSM-OpenSpace')
    task = ee.batch.Export.table.toDrive(RecSitesFC, **task_config)
    task.start()

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
out_dir = os.getcwd()

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-OSM-OpenSpace.geojson'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/open_space/openstreetmap/v_0/'+ boundary_id + '-OSM-open_space-2022.geojson',
                               ExtraArgs={'ACL':'public-read'})

# Visualize

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
i = 10
boundary_id = boundary_georef.loc[i, 'geo_name'] + '-' + boundary_georef.loc[i, 'aoi_boundary_name']
print(boundary_id)

In [ ]:
# read boundaries
boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
boundary_geo = requests.get(boundary_path).json()
boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)

In [ ]:
#read open space
openspace_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/open_space/openstreetmap/v_0/'+boundary_id+'-OSM-open_space-2022.geojson'
openspace_geo = requests.get(openspace_path).json()
openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=12)

In [ ]:
Map.addLayer(boundary_geo_ee, {'color': 'FF0000'}, 'City boundary')

In [ ]:
Map.addLayer(openspace_geo_ee,  {'color': 'FF00FF'}, 'Open space (OSM)')